In [1]:

from oxdb_lite.oxdoc.dp import DBin
import sys

In [2]:
dbin = DBin(method="byte")


In [3]:
fn = 22
#fn = "a"*1000000

In [3]:
"1".encode("utf-8")

b'1'

In [6]:

import sys


en = dbin.encode(22,ctype="n")
# ben = dbin.encode(fn,"bson")
de = dbin.decode(en)

sys.getsizeof(fn),len(en),sys.getsizeof(en),de#len(ben),sys.getsizeof(ben),

(28, 22, 55, '                 ')

In [64]:
data = {
    'data': 'string'*10,
    'int': 3010000000,
    'list': ['l1', 'l2',1,(1,1)]*10,
    'bool': True
}

In [69]:
mth = "byte"

In [70]:

ed = dbin.encode(data,mth)
rd = dbin.decode(ed,mth)
# print("encoded data : ",ed)
# print("decoded data : ",rd)
sys.getsizeof(ed)


621

In [1]:
from oxdoc.db.ld import Oxdld


doc = Oxdld("note",data_encoding="byte")

In [9]:
doc.index

{'k1': (0, 29),
 'd1': (29, 19),
 'd2': (48, 19),
 'd3': (67, 17),
 'key5': (84, 22),
 'k2': (106, 19)}

In [1]:
from oxdoc import Oxdbin

In [2]:
with open("note.oxdld/note.oxdldd.bin", "rb") as f:
    f.seek(0)
    data = f.read()
#print(data)
Oxdbin.decode_all(data)

[0,
 {'': ' dummy data-1b'},
 {'': [' dummy data-2a']},
 {'': {'data': ' dummy data-3c'}},
 {'': {'data': 'lakajsjnxubebbdiwdiwi'}},
 {'': 1},
 {'': 14},
 {'': 'dd'}]

In [ ]:
def decode_all(data_bytes:bytes):
    data = []
    data_bytes_len = len(data_bytes)
    pos = 0
    while pos<data_bytes_len :
        datai,pos = Oxdbin.decode(data_bytes,pos=pos,posless=False)
        data.append(datai)

    return data

In [2]:

def extract_data(file):
    pos = 0
    data = {"freeindex":[]}
    indict = False
    dictdata = {}
    while True:

        start_pos = pos
        file.seek(pos)
        
        # Read 1 byte, if empty, end of file
        btypeb = file.read(1)
        
        if not btypeb:
            break  # End of file

        btype = chr(btypeb[0])  # Convert byte to character
        print(btype)
        bdsize_l = Oxdbin.bdsize_len(btype=btype)
        pos =pos+ 1
        
        file.seek(pos)
        bdlenb = file.read(bdsize_l)
        if not bdlenb:
            break  # End of file

        if btype in ["i", "f"] and indict:
            data.append(Oxdbin.decode(btypeb + bdlenb))
            pos += bdsize_l

        elif btype in ["d","n"] and not indict:
            bdlen = Oxdbin.decode_bdlen(bdlenb, btype=btype)
            pos += bdsize_l
            file.seek(pos)
            
            # Read 'bdlen' bytes, if empty, end of file
            bdata = file.read(bdlen)
            if not bdata:
                break  # End of file
            print(start_pos,btype , bdlen , bdsize_l)
            print(btypeb , bdlenb , bdata)
            if btype =="n":
                data["freeindex"].append((start_pos,bdlen))
            elif btype == "d":
                #data[pos]=Oxdbin.decode( data_bytes=bdata , data_type=btype ,length=bdlen)
                indict = True
                dictdata["s"] = start_pos
                dictdata["sz"] = 1+4
                dictdata["l"]= bdlen
            pos += bdsize_l

       
        elif indict:
            bdlen = Oxdbin.decode_bdlen(bdlenb, btype=btype)
            pos += bdsize_l
            file.seek(pos)
            
            # Read 'bdlen' bytes, if empty, end of file
            bdata = file.read(bdlen)
            if not bdata:
                break  # End of file
            print(start_pos,btype , bdlen , bdsize_l)
            print(btypeb , bdlenb , bdata)
            Oxdbin.decode( data_bytes=bdata , data_type=btype ,length=bdlen)
            dictdata["sz"]+=1+4+bdlen
            pos =pos+bdlen
            data[dictdata["key"]]=(dictdata["s"],dictdata["sz"])
            indict = False

        elif( btype in ["s"] )and indict :
            
     
            bdlen = Oxdbin.decode_bdlen(bdlenb, btype=btype)
            pos += bdsize_l
            file.seek(pos)
            
            # Read 'bdlen' bytes, if empty, end of file
            bdata = file.read(bdlen)
            if not bdata:
                break  # End of file
            print(start_pos,btype , bdlen , bdsize_l)
            print(btypeb , bdlenb , bdata)
            
            key = Oxdbin.decode( data_bytes=bdata , data_type=btype ,length=bdlen)
            dictdata["sz"]+=1+4+bdlen
            pos =pos+bdlen
            data[dictdata["key"]]=(dictdata["s"],dictdata["sz"])
            indict = False
        elif( btype in ["s"] )and indict and not dictdata["key"]:
            
     
            bdlen = Oxdbin.decode_bdlen(bdlenb, btype=btype)
            pos += bdsize_l
            file.seek(pos)
            
            # Read 'bdlen' bytes, if empty, end of file
            bdata = file.read(bdlen)
            if not bdata:
                break  # End of file
            print(start_pos,btype , bdlen , bdsize_l)
            print(btypeb , bdlenb , bdata)
            
            key = Oxdbin.decode( data_bytes=bdata , data_type=btype ,length=bdlen)
            dictdata["sz"]+=1+4+bdlen
            dictdata["key"] = key

        else:
            bdlen = Oxdbin.decode_bdlen(bdlenb, btype=btype)
            pos += bdsize_l
            file.seek(pos)
            
            # Read 'bdlen' bytes, if empty, end of file
            bdata = file.read(bdlen)
            if not bdata:
                break  # End of file
            print(start_pos,btype , bdlen , bdsize_l)
            print(btypeb , bdlenb , bdata)
            Oxdbin.decode( data_bytes=bdata , data_type=btype ,length=bdlen)
            pos =pos+bdlen
            data[dictdata["key"]]=(dictdata["s"],dictdata["sz"])
            indict = False



    return data



with open("note.oxdld/note.oxdldd.bin", "rb") as file:
    print(file.read())
    print(extract_data(file=file))

b'n\x00\x00\x00\x0e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00d\x00\x00\x00\x01s\x00\x00\x00\x00s\x00\x00\x00\x0e dummy data-1bd\x00\x00\x00\x01s\x00\x00\x00\x00l\x00\x00\x00\x01s\x00\x00\x00\x0e dummy data-2ad\x00\x00\x00\x01s\x00\x00\x00\x00d\x00\x00\x00\x01s\x00\x00\x00\x04datas\x00\x00\x00\x0e dummy data-3cd\x00\x00\x00\x01s\x00\x00\x00\x00d\x00\x00\x00\x01s\x00\x00\x00\x04datas\x00\x00\x00\x15lakajsjnxubebbdiwdiwid\x00\x00\x00\x01s\x00\x00\x00\x00i\x00\x00\x00\x00\x00\x00\x00\x01d\x00\x00\x00\x01s\x00\x00\x00\x00i\x00\x00\x00\x00\x00\x00\x00\x0ed\x00\x00\x00\x01s\x00\x00\x00\x00s\x00\x00\x00\x02dd'
n
0 n 14 4
b'n' b'\x00\x00\x00\x0e' b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
d
19 d 1 4
b'd' b'\x00\x00\x00\x01' b's'


IndexError: index out of range

n                 d   s    s    dummy data-1bd   s    l   s    dummy data-


In [7]:
print(False and 6)

False


Decoding with method 'byte' failed with error: index out of range 
using other methods
Decoding with method 'json' failed with error: Expecting value: line 1 column 1 (char 0) 
using other methods
Decoding with method 'bson' failed with error: not enough data for a BSON document 
using other methods


ValueError: Failed to load data: all methods incompatible